In order to run this notebook with the correct PYTHONPATH, you can use
```
$ ./e-mission-ipy.bash notebook
```

### Preferred access technique

The preferred technique to access wrapper objects from the timeseries is to use the abstract timeseries interface. This makes it easier for us to switch to alternative timeseries implementations later

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt

In [ ]:
from uuid import UUID

In [ ]:
test_user_id = UUID("ca59292c-fd16-4463-b0ae-9bf2a4de4e52") # replace with the uuid that you loaded

In [ ]:
ts = esta.TimeSeries.get_time_series(test_user_id)

#### Accessing entries directly

In [ ]:
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=None)

All keys and their mapping to data model objects can be found in 
https://github.com/e-mission/e-mission-server/blob/master/emission/core/wrapper/entry.py

In [ ]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print "=== Trip:", cte.data.start_loc, "->", cte.data.end_loc
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print "  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode

#### Accessing a dataframe

In [ ]:
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)

In [ ]:
len(ct_df)

In [ ]:
ct_df.columns

In [ ]:
ct_df[["start_loc", "end_loc", "start_ts", "end_ts"]]

In [ ]:
cs_df = ts.get_data_df("analysis/cleaned_section", time_query=None)

In [ ]:
len(cs_df)

In [ ]:
cs_df.columns

In [ ]:
cs_df[["start_loc", "end_loc", "start_ts", "end_ts", "sensed_mode"]]

### Direct mongodb queries

You can also use direct mongodb queries during exploratory work. I do ask that you create a storage decorator (`emission/storage/decorations`) when you submit a pull request for ongoing use

In [ ]:
import emission.core.get_database as edb

In [ ]:
edb.get_timeseries_db().find_one()

In [ ]:
edb.get_timeseries_db().distinct("metadata.key")

Note that in this case, you need to know whether to use the `timeseries` or the `analysis_timeseries` collection

In [ ]:
edb.get_analysis_timeseries_db().distinct("metadata.key")

In [ ]:
edb.get_analysis_timeseries_db().find({"user_id": test_user_id, "metadata.key": "analysis/cleaned_trip"}).count()

In particular, you can use this to access entries that are not in the timeseries

In [ ]:
edb.get_uuid_db().distinct("uuid")

### Timeline

The trips and places maintain links to each other - e.g. `start_place`, `end_place`

In [ ]:
ct_df[["start_place", "end_place"]]

These are _primary key links_ to other entries in the database. It would be useful to have a doubly linked list representing this properly. The Timeline helps with that.

In [ ]:
import emission.storage.decorations.timeline as esdl

In [ ]:
tl = esdl.get_cleaned_timeline(test_user_id, ct_df.iloc[0].start_ts, ct_df.iloc[-1].end_ts)

In [ ]:
for e in tl:
    print e.metadata.key

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.first_trip().get_id())

In [ ]:
for e in stl:
    print e.metadata.key

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.last_trip().get_id())

In [ ]:
for e in stl:
    print e.metadata.key

## Getting trip and section details ##

Once we have trip and section objects, we can retrieve the sensed data associated with them by querying for data in various streams that falls within the time ranges associated with the trip/section. Here again, our architecture of storing the analysis results as a separate datastream makes it easy to retrieve data at various levels of processing.


### Get locations with no processing, basic filtering and resampling for the first trip ###

In [ ]:
all_locs = ts.get_data_df("background/location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

filtered_locs = ts.get_data_df("background/filtered_location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

resampled_locs = ts.get_data_df("analysis/recreated_location",
                                 time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))
print("Locations go from all = %d -> filtered = %d -> resampled = %d" % (len(all_locs),
                                                                         len(filtered_locs),
                                                                         len(resampled_locs)))

### Get the raw motion activity, in case you want to do different segmentation ###

In [ ]:
all_activity = ts.get_data_df("background/motion_activity",
                               esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

In [ ]:
all_activity.columns

In [ ]:
import emission.core.wrapper.motionactivity as ecwm

print("Found %d walking entries, %d on_foot entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.WALKING.value]), 
                                    len(all_activity[all_activity.type == ecwm.MotionTypes.ON_FOOT.value])))
print("Found %d motorized entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.IN_VEHICLE.value])))

### Plot the location points ###

For this to work, you need a custom version of folium which supports my custom geojson plugin.
This version is available at http://cs.berkeley.edu/~shankari/folium.zip
and needs to be unzipped into the e-mission-server directory.

TODO: Switch to the current version of folium, which supposedly supports geojson out of the box instead of this hacky solution.

In [ ]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.ipython_helper as ipy
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(all_locs)])])

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(all_locs),
                                                      gfc.get_feature_list_from_df(filtered_locs),
                                                      gfc.get_feature_list_from_df(resampled_locs)])])